In [233]:
import pandas as pd
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True)

In [234]:
data_breast_cancer.data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [235]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_breast_cancer.data.loc[:, ["mean texture", "mean symmetry"]], data_breast_cancer.target, test_size=0.2)
import pickle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [236]:
log_clf = LogisticRegression(solver="lbfgs",
                             random_state=42)
dctree_clf =  DecisionTreeClassifier()

neigh_clf = KNeighborsClassifier()
first = []
first_classificators = []
voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf),
                ('rf', dctree_clf),
                ('knn', neigh_clf)],
    voting='hard')

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf),
                ('rf', dctree_clf),
                ('knn', neigh_clf)],
    voting='soft')
for clf in ( dctree_clf, log_clf,neigh_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    first_classificators.append(clf)
    first.append((accuracy_score(y_train, y_pred_train),accuracy_score(y_test, y_pred)))
    


In [237]:
pickle.dump(first, open("acc_vote.pkl", "wb"))
pickle.dump(first_classificators, open("vote.pkl", "wb"))

In [238]:
acc_bag = []
from sklearn.ensemble import BaggingClassifier



bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=100, bootstrap=True)
bag_clf.fit(X_train, y_train)
acc_bag.append((bag_clf.score(X_train, y_train),bag_clf.score(X_test, y_test)))



In [239]:
bag_clf_zero = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=0.5, bootstrap=True)
bag_clf_zero.fit(X_train, y_train)
acc_bag.append((bag_clf_zero.score(X_train, y_train),bag_clf_zero.score(X_test, y_test)))

In [240]:
bag_clf_first = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=100, bootstrap=False)
bag_clf_first.fit(X_train, y_train)
acc_bag.append((bag_clf_first.score(X_train, y_train),bag_clf_first.score(X_test, y_test)))

In [241]:
bag_clf_second = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=30,
max_samples=0.5, bootstrap=False)
bag_clf_second.fit(X_train, y_train)
acc_bag.append((bag_clf_second.score(X_train, y_train),bag_clf_second.score(X_test, y_test)))

In [242]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500,
                                 max_leaf_nodes=16)
rnd_clf.fit(X_train, y_train)
acc_bag.append((rnd_clf.score(X_train, y_train),rnd_clf.score(X_test, y_test)))

In [243]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, data_breast_cancer.data.loc[:, ["mean texture", "mean symmetry"]], data_breast_cancer.target, cv=5)
acc_bag.append((scores.mean(), scores.mean()))

In [244]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(max_depth=2
                                 ,n_estimators=3,learning_rate=1.0)
gbrt.fit(X_train, y_train)
acc_bag.append((gbrt.score(X_train, y_train),gbrt.score(X_test, y_test)))

In [245]:
bag = [bag_clf, bag_clf_zero, bag_clf_first, bag_clf_second, rnd_clf, clf, gbrt]

In [246]:
pickle.dump(acc_bag, open("acc_bag.pkl", "wb"))
pickle.dump(bag, open("bag.pkl", "wb"))

In [247]:

bag_clf_fea = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=30,
    max_samples=100, bootstrap=True, max_features=2)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(data_breast_cancer.data, data_breast_cancer.target, test_size=0.2)
bag_clf_fea.fit(X_train_1, y_train_1)
acc_fea_list = [bag_clf_fea.score(X_train_1, y_train_1), bag_clf_fea.score(X_test_1, y_test_1)]

In [248]:
pickle.dump(acc_fea_list, open("acc_fea.pkl", "wb"))
pickle.dump([bag_clf_fea], open("fea.pkl", "wb"))

In [249]:
acc_fea_rank_list_acc_train = []
acc_fea_rank_list_acc_test = []
acc_fea_rank_list_names = []
for x, clf in enumerate(bag_clf_fea.estimators_):
    a , b = data_breast_cancer.data.columns[bag_clf_fea.estimators_features_[x][0]], data_breast_cancer.data.columns[bag_clf_fea.estimators_features_[x][1]]
    location = lambda y:  y.loc[:,[a,b]]

    y_pred = clf.predict(location(X_test_1))
    y_pred_train = clf.predict(location(X_train_1))
    acc_fea_rank_list_acc_train.append(accuracy_score(y_train_1.values, y_pred_train))
    acc_fea_rank_list_acc_test.append(accuracy_score(y_test_1.values, y_pred))
    acc_fea_rank_list_names.append([a,b])

In [250]:
data = {'Train accuracy':acc_fea_rank_list_acc_train, 'Test accuracy':acc_fea_rank_list_acc_test, "feature_names":acc_fea_rank_list_names}
df_acc = pd.DataFrame(data)
df_acc

,Train accuracy,Test accuracy,feature_names
0,0.890110,0.824561,"[mean radius, mean fractal dimension]"
1,0.819780,0.745614,"[worst fractal dimension, perimeter error]"
2,0.718681,0.622807,"[worst texture, symmetry error]"
3,0.804396,0.745614,"[perimeter error, smoothness error]"
4,0.784615,0.719298,"[radius error, compactness error]"
5,0.863736,0.798246,"[mean symmetry, mean perimeter]"
6,0.920879,0.868421,"[area error, worst concave points]"
7,0.694505,0.561404,"[fractal dimension error, symmetry error]"
8,0.945055,0.929825,"[mean texture, worst perimeter]"
9,0.890110,0.868421,"[concave points error, mean perimeter]"


In [251]:
df_acc = df_acc.sort_values(by=['Train accuracy', 'Test accuracy'], ascending=False)
df_acc

,Train accuracy,Test accuracy,feature_names
8,0.945055,0.929825,"[mean texture, worst perimeter]"
28,0.931868,0.877193,"[worst perimeter, concavity error]"
25,0.927473,0.885965,"[worst perimeter, worst compactness]"
23,0.925275,0.850877,"[worst radius, worst area]"
29,0.923077,0.885965,"[worst perimeter, mean fractal dimension]"
6,0.920879,0.868421,"[area error, worst concave points]"
12,0.892308,0.815789,"[radius error, worst concavity]"
9,0.890110,0.868421,"[concave points error, mean perimeter]"
0,0.890110,0.824561,"[mean radius, mean fractal dimension]"
18,0.890110,0.824561,"[mean texture, area error]"


In [252]:

pickle.dump(df_acc, open("acc_fea_rank.pkl", "wb"))